In [1]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
import time
import pandas as pd

    ## URL creation for webscrapping 
    eg : https://www.indeed.com/jobs?q=%22Data+Engineer%22(python+or+pandas)&l=Oregon&radius=0&sort=date
    to make the search more efficient, tried to use skills and job title so that it would give better results.
    
    Did try the approach to run multiple urls in a for loop so that we could use a list of job titles for a state, 
    but it would hang the web scrapper , so for this project used one title and one state at a time.
    
    Better approach could have been to take the job title and State as input parameter but not done for this project.

In [2]:
#Obtaining an instance of browser
browser = webdriver.Chrome('windows/chromedriver')
#job_type ("Data+Scientist" or "Data+Engineer" or "Data+Analyst"]
#job_state ("Washington" or "New+York" or "Massachusetts" or "Ohio" or "California")

#Variable declarion
job_type = "Data+Engineer"
job_state = "Oregon"
skills = "(python+or+pandas)"
job_criteria = skills+job_type
#URL Created 
#https://www.indeed.com/jobs?q=%22data+scientist%22+(python+or+pandas)&l=OH&radius=0&sort=date&fromage=30
url = "https://www.indeed.com/jobs?q=%22" + job_type + "%22"+skills+"&l="+job_state+"&radius=0&sort=date"
print(url)
browser.get(url)
#Need to add sleep time so that the code and selenium work fine.
time.sleep(1)

https://www.indeed.com/jobs?q=%22Data+Engineer%22(python+or+pandas)&l=Oregon&radius=0&sort=date


In [3]:
#Creating a List to hold all the job openings here
job_opening = []
x = 0
scrapping_bool = True
#While there is a data to scrape this loop will run
while scrapping_bool:
    #trying to keep a count of pages read
    x +=1
    # HTML object
    html = browser.page_source
    # Parse HTML with Beautiful Soup
    soup = BeautifulSoup(html, 'html.parser')
    
    # Retrieve all elements that have job opening information
    job_listing = soup.find_all('div', class_='jobsearch-SerpJobCard')
    #print(f"this is the no of job listing on webpage {x}")
   # print(f"this is the no of job listing obtained {len(job_title)}")

    for listing in job_listing:
        
        title_detail = listing.find('div',class_='title')
        title = title_detail.a['title']
        
        company_detail = listing.find('div',class_='sjcl')
        company_name = company_detail.span.text.strip()
        
        handle_loc = listing.find('div',class_='recJobLoc')
        location = handle_loc['data-rc-loc'].strip()
        job_summary = listing.find('div',class_='summary').text.strip()
        #Creating a dictionary object for each job opening   
        dict_jobopening = {"job_title": title,"company_name": company_name,"location":location,
                          "job_summary": job_summary}#,"reviews": reviews,"ratings": ratings}
        job_opening.append(dict_jobopening)
        
        time.sleep(1)
       
    try:
        #alert = browser.switch_to_alert()
        #alert.dismiss()
        #To click on the pop-ups that keeps popping up on every page almost
        browser.find_element_by_partial_link_text('No, thanks').click() 
        time.sleep(1)

    except:
        print("No Pop-Up Found.")

    try:
        time.sleep(1)
        print(f"Lenght of list is {len(job_opening)}")
        time.sleep(1)
        print(company_name)
        #Clicking the next link, checking if the word is not a match in the name of the company eg Nextdoor
        nextElem = browser.find_element_by_partial_link_text('Next')
        #print(nextElem.text)
        if(nextElem.text == "Next »"):
            nextElem.click()
            print(f"Clicking Next {x}")
            time.sleep(1)
     
    except NoSuchElementException:

        print("Scraping Complete")
        scrapping_bool = False
        break
            
#closing the browser window    
browser.close()


No Pop-Up Found.
Lenght of list is 14
AccruePartners
Clicking Next 1
Lenght of list is 27
XLFN Consulting
Clicking Next 2
No Pop-Up Found.
Lenght of list is 32
Slalom Consulting
No Pop-Up Found.
Lenght of list is 37
Slalom Consulting
No Pop-Up Found.
Lenght of list is 42
Slalom Consulting
No Pop-Up Found.
Lenght of list is 47
Slalom Consulting
No Pop-Up Found.
Lenght of list is 52
Slalom Consulting
No Pop-Up Found.
Lenght of list is 57
Slalom Consulting
No Pop-Up Found.
Lenght of list is 62
Slalom Consulting
No Pop-Up Found.
Lenght of list is 67
Slalom Consulting
No Pop-Up Found.
Lenght of list is 72
Slalom Consulting
No Pop-Up Found.
Lenght of list is 77
Slalom Consulting
No Pop-Up Found.
Lenght of list is 82
Slalom Consulting
No Pop-Up Found.
Lenght of list is 87
Slalom Consulting
No Pop-Up Found.
Lenght of list is 92
Slalom Consulting
No Pop-Up Found.
Lenght of list is 97
Slalom Consulting
No Pop-Up Found.
Lenght of list is 102
Slalom Consulting
No Pop-Up Found.
Lenght of list is 10

No Pop-Up Found.
Lenght of list is 742
Slalom Consulting
No Pop-Up Found.
Lenght of list is 747
Slalom Consulting
No Pop-Up Found.
Lenght of list is 752
Slalom Consulting
No Pop-Up Found.
Lenght of list is 757
Slalom Consulting
No Pop-Up Found.
Lenght of list is 762
Slalom Consulting
No Pop-Up Found.
Lenght of list is 767
Slalom Consulting
No Pop-Up Found.
Lenght of list is 772
Slalom Consulting
No Pop-Up Found.
Lenght of list is 777
Slalom Consulting
No Pop-Up Found.
Lenght of list is 782
Slalom Consulting
No Pop-Up Found.
Lenght of list is 787
Slalom Consulting
No Pop-Up Found.
Lenght of list is 792
Slalom Consulting
No Pop-Up Found.
Lenght of list is 797
Slalom Consulting
No Pop-Up Found.
Lenght of list is 802
Slalom Consulting
No Pop-Up Found.
Lenght of list is 807
Slalom Consulting
No Pop-Up Found.
Lenght of list is 812
Slalom Consulting
No Pop-Up Found.
Lenght of list is 817
Slalom Consulting
No Pop-Up Found.
Lenght of list is 822
Slalom Consulting
No Pop-Up Found.
Lenght of list

Lenght of list is 1452
Slalom Consulting
No Pop-Up Found.
Lenght of list is 1457
Slalom Consulting
No Pop-Up Found.
Lenght of list is 1462
Slalom Consulting
No Pop-Up Found.
Lenght of list is 1467
Slalom Consulting
No Pop-Up Found.
Lenght of list is 1472
Slalom Consulting
No Pop-Up Found.
Lenght of list is 1477
Slalom Consulting
No Pop-Up Found.
Lenght of list is 1482
Slalom Consulting
No Pop-Up Found.
Lenght of list is 1487
Slalom Consulting
No Pop-Up Found.
Lenght of list is 1492
Slalom Consulting
No Pop-Up Found.
Lenght of list is 1497
Slalom Consulting
No Pop-Up Found.
Lenght of list is 1502
Slalom Consulting
No Pop-Up Found.
Lenght of list is 1507
Slalom Consulting
No Pop-Up Found.
Lenght of list is 1512
Slalom Consulting
No Pop-Up Found.
Lenght of list is 1517
Slalom Consulting
No Pop-Up Found.
Lenght of list is 1522
Slalom Consulting
No Pop-Up Found.
Lenght of list is 1527
Slalom Consulting
No Pop-Up Found.
Lenght of list is 1532
Slalom Consulting
No Pop-Up Found.
Lenght of list

Slalom Consulting
No Pop-Up Found.
Lenght of list is 2162
Slalom Consulting
No Pop-Up Found.
Lenght of list is 2167
Slalom Consulting
No Pop-Up Found.
Lenght of list is 2172
Slalom Consulting
No Pop-Up Found.
Lenght of list is 2177
Slalom Consulting
No Pop-Up Found.
Lenght of list is 2182
Slalom Consulting
No Pop-Up Found.
Lenght of list is 2187
Slalom Consulting
No Pop-Up Found.
Lenght of list is 2192
Slalom Consulting
No Pop-Up Found.
Lenght of list is 2197
Slalom Consulting
No Pop-Up Found.
Lenght of list is 2202
Slalom Consulting
No Pop-Up Found.
Lenght of list is 2207
Slalom Consulting
No Pop-Up Found.
Lenght of list is 2212
Slalom Consulting
No Pop-Up Found.
Lenght of list is 2217
Slalom Consulting
No Pop-Up Found.
Lenght of list is 2222
Slalom Consulting
No Pop-Up Found.
Lenght of list is 2227
Slalom Consulting
No Pop-Up Found.
Lenght of list is 2232
Slalom Consulting
No Pop-Up Found.
Lenght of list is 2237
Slalom Consulting
No Pop-Up Found.
Lenght of list is 2242
Slalom Consult

No Pop-Up Found.
Lenght of list is 2867
Slalom Consulting
No Pop-Up Found.
Lenght of list is 2872
Slalom Consulting
No Pop-Up Found.
Lenght of list is 2877
Slalom Consulting
No Pop-Up Found.
Lenght of list is 2882
Slalom Consulting
No Pop-Up Found.
Lenght of list is 2887
Slalom Consulting
No Pop-Up Found.
Lenght of list is 2892
Slalom Consulting
No Pop-Up Found.
Lenght of list is 2897
Slalom Consulting
No Pop-Up Found.
Lenght of list is 2902
Slalom Consulting
No Pop-Up Found.
Lenght of list is 2907
Slalom Consulting
No Pop-Up Found.
Lenght of list is 2912
Slalom Consulting
No Pop-Up Found.
Lenght of list is 2917
Slalom Consulting
No Pop-Up Found.
Lenght of list is 2922
Slalom Consulting
No Pop-Up Found.
Lenght of list is 2927
Slalom Consulting
No Pop-Up Found.
Lenght of list is 2932
Slalom Consulting
No Pop-Up Found.
Lenght of list is 2937
Slalom Consulting
No Pop-Up Found.
Lenght of list is 2942
Slalom Consulting
No Pop-Up Found.
Lenght of list is 2947
Slalom Consulting
No Pop-Up Foun

...

Slalom Consulting
No Pop-Up Found.
Lenght of list is 6402
Slalom Consulting
No Pop-Up Found.
Lenght of list is 6407
Slalom Consulting
No Pop-Up Found.
Lenght of list is 6412
Slalom Consulting
No Pop-Up Found.
Lenght of list is 6417
Slalom Consulting
No Pop-Up Found.
Lenght of list is 6422
Slalom Consulting
No Pop-Up Found.
Lenght of list is 6427
Slalom Consulting
No Pop-Up Found.
Lenght of list is 6432
Slalom Consulting
No Pop-Up Found.
Lenght of list is 6437
Slalom Consulting
No Pop-Up Found.
Lenght of list is 6442
Slalom Consulting
No Pop-Up Found.
Lenght of list is 6447
Slalom Consulting
No Pop-Up Found.
Lenght of list is 6452
Slalom Consulting
No Pop-Up Found.
Lenght of list is 6457
Slalom Consulting
No Pop-Up Found.
Lenght of list is 6462
Slalom Consulting
No Pop-Up Found.
Lenght of list is 6467
Slalom Consulting
No Pop-Up Found.
Lenght of list is 6472
Slalom Consulting
No Pop-Up Found.
Lenght of list is 6477
Slalom Consulting
No Pop-Up Found.
Lenght of list is 6482
Slalom Consult

No Pop-Up Found.
Lenght of list is 7107
Slalom Consulting
No Pop-Up Found.
Lenght of list is 7112
Slalom Consulting
No Pop-Up Found.
Lenght of list is 7117
Slalom Consulting
No Pop-Up Found.
Lenght of list is 7122
Slalom Consulting
No Pop-Up Found.
Lenght of list is 7127
Slalom Consulting
No Pop-Up Found.
Lenght of list is 7132
Slalom Consulting
No Pop-Up Found.
Lenght of list is 7137
Slalom Consulting
No Pop-Up Found.
Lenght of list is 7142
Slalom Consulting
No Pop-Up Found.
Lenght of list is 7147
Slalom Consulting
No Pop-Up Found.
Lenght of list is 7152
Slalom Consulting
No Pop-Up Found.
Lenght of list is 7157
Slalom Consulting
No Pop-Up Found.
Lenght of list is 7162
Slalom Consulting
No Pop-Up Found.
Lenght of list is 7167
Slalom Consulting
No Pop-Up Found.
Lenght of list is 7172
Slalom Consulting
No Pop-Up Found.
Lenght of list is 7177
Slalom Consulting
No Pop-Up Found.
Lenght of list is 7182
Slalom Consulting
No Pop-Up Found.
Lenght of list is 7187
Slalom Consulting
No Pop-Up Foun

Lenght of list is 7812
Slalom Consulting
No Pop-Up Found.
Lenght of list is 7817
Slalom Consulting
No Pop-Up Found.
Lenght of list is 7822
Slalom Consulting
No Pop-Up Found.
Lenght of list is 7827
Slalom Consulting
No Pop-Up Found.
Lenght of list is 7832
Slalom Consulting
No Pop-Up Found.
Lenght of list is 7837
Slalom Consulting
No Pop-Up Found.
Lenght of list is 7842
Slalom Consulting
No Pop-Up Found.
Lenght of list is 7847
Slalom Consulting
No Pop-Up Found.
Lenght of list is 7852
Slalom Consulting
No Pop-Up Found.
Lenght of list is 7857
Slalom Consulting
No Pop-Up Found.
Lenght of list is 7862
Slalom Consulting
No Pop-Up Found.
Lenght of list is 7867
Slalom Consulting
No Pop-Up Found.
Lenght of list is 7872
Slalom Consulting
No Pop-Up Found.
Lenght of list is 7877
Slalom Consulting
No Pop-Up Found.
Lenght of list is 7882
Slalom Consulting
No Pop-Up Found.
Lenght of list is 7887
Slalom Consulting
No Pop-Up Found.
Lenght of list is 7892
Slalom Consulting
No Pop-Up Found.
Lenght of list

Slalom Consulting
No Pop-Up Found.
Lenght of list is 8522
Slalom Consulting
No Pop-Up Found.
Lenght of list is 8527
Slalom Consulting
No Pop-Up Found.
Lenght of list is 8532
Slalom Consulting
No Pop-Up Found.
Lenght of list is 8537
Slalom Consulting
No Pop-Up Found.
Lenght of list is 8542
Slalom Consulting
No Pop-Up Found.
Lenght of list is 8547
Slalom Consulting
No Pop-Up Found.
Lenght of list is 8552
Slalom Consulting
No Pop-Up Found.
Lenght of list is 8557
Slalom Consulting
No Pop-Up Found.
Lenght of list is 8562
Slalom Consulting
No Pop-Up Found.
Lenght of list is 8567
Slalom Consulting
No Pop-Up Found.
Lenght of list is 8572
Slalom Consulting
No Pop-Up Found.
Lenght of list is 8577
Slalom Consulting
No Pop-Up Found.
Lenght of list is 8582
Slalom Consulting
No Pop-Up Found.
Lenght of list is 8587
Slalom Consulting
No Pop-Up Found.
Lenght of list is 8592
Slalom Consulting
No Pop-Up Found.
Lenght of list is 8597
Slalom Consulting
No Pop-Up Found.
Lenght of list is 8602
Slalom Consult

WebDriverException: Message: chrome not reachable
  (Session info: chrome=78.0.3904.108)


## Creating a Dataframe object out of the scrapped data

In [ ]:
job_opening_df = pd.DataFrame(job_opening)
job_opening_df.head()




## Splitting the location column to state and city

In [ ]:
# new data frame with split value columns 
new = job_opening_df["location"].str.split(",", n = 1, expand = True) 
#print(new)  
# making separate first name column from new data frame 
job_opening_df["City"]= new[0] 
  
# making separate last name column from new data frame 
job_opening_df["State"]= new[1] 
  
# Dropping old Name columns 
job_opening_df.drop(columns =["location"], inplace = True) 

In [ ]:
len(job_opening_df)

## Writing the data to an excel file, and then post that manually changing the excel to UTF8 CSV file.

In [ ]:
outFileName = job_state+"_output"+job_type+".xlsx"
print(outFileName)
job_opening_df.to_excel("output/"+outFileName)  
#job_opening_df.to_csv("output/"+outFileName, sep='\t', index=False, header=True, encoding='utf-8')